# Data download

In [ ]:
!pip install curl unzip
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/Agentic+Chatbot+Assurance+Habitation+-+Processus.md data/
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/Agentic+Chatbot+Assurance+Habitation+-+Garanties.md data/
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/attachments.zip data/
!unzip data/attachments.zip data/
!rm data/attachments.zip

In [16]:

import re
import sys
from pathlib import Path

from typing import AsyncIterator
from typing_extensions import TypedDict
from typing import Callable

from langgraph.types import interrupt
from langgraph.graph import StateGraph, START, END


# Declaration agent

## Tools
2. parse_declaration
3. verify_completeness
4. ask_human


In [13]:
class DeclarationReActState(TypedDict):
    question: str  # La question initiale de l'utilisateur
    history: list[str]  # L'historique des échanges (Thought, Action, Observation)
    last_action: str | None  # Le nom de l'outil à appeler (si applicable)
    last_arguments: dict | None  # Les arguments à passer à l'outil
    last_observation: str | None  # Le résultat de l'outil appelé
    is_complete: str | None  # La réponse finale
    parsed_declaration: dict | None
    missing: dict | None

In [15]:
from assurhabitat_agents.tools.parse_declaration_tool import parse_declaration
from assurhabitat_agents.tools.ask_human_tool import ask_human
from assurhabitat_agents.tools.verify_completness_tool import verify_completeness

tools = {
    "DeclarationParser": parse_declaration,
    "AskHuman": ask_human,
    "InformationVerification": verify_completeness
}

actions = "\n\n".join([
    f"### Action `{t}`\n\n{tools[t].__doc__}"
    for t in tools
])
print(actions)

ImportError: cannot import name 'llm_inference' from 'assurhabitat_agents.llm.model_loading' (/Users/kraligan/assurhabitat_agents/src/assurhabitat_agents/llm/model_loading.py)

## Nodes
1. thought
2. execute

In [ ]:
def format_prompt(history: list[str]) -> str:
    """Template de prompt pour l'agent ReAct."""
    prompt = """
Tu es un agent IA qui utilise des étapes de réflexion et action pour répondre à des questions. Pour chaque question, tu dois fournir une réflexion et une action associée.

Voici les actions disponibles.

{actions}

Voici quelques exemples de réflexions et d'actions.

## Exemple

### Entrée

Question: Bonjour, on m'a cambriolé ce matin, les voleurs sont passés par le vélux de la 
chambre et ont volé tous les appareils électroniques. Merci de me contacter rapidement.
Réflexion: J'ai besoin de parser cette declaration
Action: DeclarationParser
Arguments:
  raw_input=Question
Observation: {...}

### Sortie

Réflexion: J'ai besoin de verifier si ces informations sont completes.
Action: InformationVerification
Arguments:
  parsed_declaration={...}

(Ajouter des exemple avec en entree un declaration incomplete qui conduit a AskHuman, puis un exemple )


## Règles

- Si tu ne trouves pas d'outil approprié, ne fournis pas d'action et donne simplement la réponse.
- Tu dois fournir la réponse finale dans la langue de la question d'origine.
- Lorsque tu fournis des arguments, tu dois les définir sous la forme `nom_argument=valeur_argument`, avec une ligne par argument.
- Tu ne dois pas ré-écrire dans la sortie, ce qu'il y a en entrée.
- Tu dois juste afficher la sortie qui est censé être la suite du raisonnement entrée : tu ne dois avoir qu'une seule paire réflexion/action, ou directement la réponse.

## Inférence

Propose maintenant une nouvelle réflexion et une nouvelle action et/ou réponse pour l'historique suivant.

### Entrée

{history}

### Sortie

""".strip()
    return prompt.format(actions=actions, history="\n".join([x.strip() for x in history]))

In [ ]:
def parse_output(output: str):
    action_match = re.search(r"Action: (.+)", output)  # On récupère le nom de l'outil
    arguments_match = re.search(r"Arguments:([\s\S]*)", output)  # On récupère les arguments
    answer_match = re.search(r"Réponse: (.*)", output)

    if action_match: # Si l'agent propose un outil, alors on l'utilise en priorité
        tool_name = action_match.group(1)
        tool_input = {}
        if arguments_match is not None:  # Dans ce cas, on dispose au moins d'un argument
            tool_input = {}
            for arg in arguments_match.group(1).split("\n"):
                if len(arg.strip()) == 0:  # Si le texte est vide, on ignore
                    continue
                argument_re = re.search(r"^\s*(.*)\=(.*)$", arg)  # On récupère d'un part le nom de l'argument, et d'autre part sa valeur
                if argument_re is None or argument_re is not None and len(argument_re.groups()) != 2:
                    raise ValueError(f"Impossible to parse argument '{arg}' for action '{tool_name}'.")
                tool_input[argument_re.group(1)] = argument_re.group(2)  # Ensuite, on récupère l'argument et sa valeur (tout sera donc en str)
        return "action", tool_name, tool_input
    elif answer_match:
        return "answer", answer_match.group(1).strip()
    else:
        return "thought", output.strip()

def node_thought_action(state: ReActState) -> ReActState:
    """Noeud qui va déterminer la prochaine action à réaliser."""
    # On formate le prompt à partir de l'historique
    prompt = format_prompt(state["history"])
    output = llm_inference(prompt)
    # On parse la sortie pour détecter action, arguments, ou answer
    step_type, *content = parse_output(output)
    if step_type == "action":
        tool_name, tool_input = content
        state["last_action"] = tool_name
        state["last_arguments"] = tool_input
        state["history"].append(output)
    elif step_type == "answer":
        state["answer"] = content[0]
        state["history"].append(output)
    else:
        # Thought seul, on l'ajoute à l'historique et on reboucle
        state["history"].append(output)
    return state

def node_tool_execution(state: ReActState) -> ReActState:
    """Noeud qui va exécuter un outil et modifier l'état avec l'observation obtenue."""
    tool_name = state["last_action"]
    tool_args = state["last_arguments"] or {}
    if tool_name in tools:
        try:
            observation = tools[tool_name](**tool_args)
        except Exception as e:
            observation = f"Error: {e}"
    else:
        observation = f"Error: Unknown tool {tool_name}"
    state["last_observation"] = str(observation)
    state["history"].append(f"Observation: {observation}")
    # On réinitialise l'action pour la prochaine itération
    state["last_action"] = None
    state["last_arguments"] = None
    return state

## Graph building